In [37]:
import numpy as np
import random

In [38]:


def polynomial_remainder(dividend, divisor):
    current_remainder = np.copy(dividend)
    len_divisor = len(divisor)
    while len(current_remainder) >= len_divisor:
        degree_shift = len(current_remainder) - len_divisor
        current_remainder[degree_shift:degree_shift + len_divisor] = np.bitwise_xor(current_remainder[degree_shift:degree_shift + len_divisor], divisor)
        while len(current_remainder) > 0 and current_remainder[-1] == 0:
            current_remainder = current_remainder[:-1]

    return current_remainder


def polynomial_multiply(poly1, poly2):

    result_size = len(poly1) + len(poly2) - 1
    product = np.zeros(result_size, dtype=int)

    for j in range(len(poly2)):
        if poly2[j]:
            for i in range(len(poly1)):
                product[j + i] ^= poly1[i]

    return product

def error_and_correction(original_message, generator, error_count):

    transmitted_word = polynomial_multiply(original_message, generator)
    print("Отправленное слово: ", transmitted_word)


    received_word = transmitted_word.copy()
    error_pattern = np.zeros(len(received_word), dtype=int)


    if error_count == 1:
        error_position = random.randint(0, len(received_word) - 1)
        error_pattern[error_position] = 1
    elif error_count == 2:
        pos1 = random.randint(0, len(received_word) - 2)
        pos2 = (pos1 + random.choice([1, 2])) % len(received_word)
        error_pattern[pos1] = error_pattern[pos2] = 1
    else:
        error_indices = random.sample(range(len(received_word)), error_count)
        error_pattern[error_indices] = 1


    received_word = (received_word + error_pattern) % 2
    print("Слово с ошибками:", received_word)


    syndrome = polynomial_remainder(received_word, generator)


    error_templates = [[1]] if error_count == 1 else [[1, 1, 1], [1, 0, 1], [1, 1], [1]]


    index = 0
    is_match_found = False

    while not is_match_found:
        if any(np.array_equal(syndrome, template) for template in error_templates) or (index > len(received_word) - 1):
            is_match_found = True
        else:
            syndrome = polynomial_remainder(polynomial_multiply(syndrome, np.array([0, 1])), generator)
            index += 1


    correction_pattern = np.zeros(len(received_word), dtype=int)
    if index == 0:
        correction_pattern[index] = 1
    else:
        correction_pattern[len(correction_pattern) - index] = 1

    error_correction = polynomial_multiply(syndrome, correction_pattern)[:len(received_word)]
    corrected_message = (received_word + error_correction) % 2
    print("Исправленное слово: ", corrected_message)

    if np.array_equal(transmitted_word, corrected_message):
        print("Успешно: ", corrected_message)
    else:
        print("Ошибка в исправлении", corrected_message)




In [39]:
def check(a,g,count_mistake):

  match count_mistake:
    case 1:
      print("1. Исправление для однократной ошибки")
      error_and_correction(a, g, 1)
    case 2:
      print("2. Исправление для двухкратной ошибки")
      error_and_correction(a, g, 2)
    case 3:
      print("3. Исправление для трехкратной ошибки")
      error_and_correction(a, g, 3)
    case 4:
      print("4. Исправление для четырехкратной ошибки")
      error_and_correction(a, g, 4)


6.1 Написать функции кодирования и декодирования для циклического
кода
порождающим многочленом 𝑔(𝑥) = 1+𝑥2 +𝑥3,
исправляющего однократные ошибки и провести исследование этого кода
для одно-, двух- и трёхкратных ошибок.6/

In [40]:
a = np.array([1, 0, 0, 1])
g = np.array([1, 0, 1, 1])


print("Порождающий полином:", g)
print("Входное слово:", a)


check(a, g, 1)
check(a, g, 2)
check(a, g, 3)

Порождающий полином: [1 0 1 1]
Входное слово: [1 0 0 1]
1. Исправление для однократной ошибки
Отправленное слово:  [1 0 1 0 0 1 1]
Слово с ошибками: [1 0 1 0 0 0 1]
Исправленное слово:  [1 0 1 0 0 1 1]
Успешно:  [1 0 1 0 0 1 1]
2. Исправление для двухкратной ошибки
Отправленное слово:  [1 0 1 0 0 1 1]
Слово с ошибками: [0 1 1 0 0 1 1]
Исправленное слово:  [1 0 1 0 0 1 1]
Успешно:  [1 0 1 0 0 1 1]
3. Исправление для трехкратной ошибки
Отправленное слово:  [1 0 1 0 0 1 1]
Слово с ошибками: [1 1 1 0 1 1 0]
Исправленное слово:  [0 0 1 0 1 1 0]
Ошибка в исправлении [0 0 1 0 1 1 0]


6.2 Написать функции кодирования и декодирования для циклического
кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1+ 𝑥3 +𝑥4 +𝑥5 +𝑥6,
исправляющего пакеты ошибок кратности 3 и провести исследование

In [41]:
a = np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])
g = np.array([1, 0, 0, 1, 1, 1, 1])

print("Порождающий полином:", g)
print("Входное слово:", a)


check(a, g, 1)
check(a, g, 2)
check(a, g, 3)
check(a, g, 4)

Порождающий полином: [1 0 0 1 1 1 1]
Входное слово: [1 0 0 1 0 0 0 1 1]
1. Исправление для однократной ошибки
Отправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Слово с ошибками: [0 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Исправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Успешно:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
2. Исправление для двухкратной ошибки
Отправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Слово с ошибками: [1 0 0 0 1 1 0 0 0 1 1 0 0 1 0]
Исправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Успешно:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
3. Исправление для трехкратной ошибки
Отправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Слово с ошибками: [1 0 0 0 1 1 0 0 1 1 1 0 1 1 1]
Исправленное слово:  [0 1 1 0 1 1 0 0 1 1 1 0 1 1 1]
Ошибка в исправлении [0 1 1 0 1 1 0 0 1 1 1 0 1 1 1]
4. Исправление для четырехкратной ошибки
Отправленное слово:  [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Слово с ошибками: [1 0 1 0 1 1 0 0 1 0 0 0 0 0 1]
Исправленное слово:  [1 0 1 0 1 1 0 0 1 0 0 0 0 1 1]
Ошибка в исп